In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt

from rasterio.transform import from_origin
from rasterio.plot import show

from potentiel_solaire.constants import RESULTS_FOLDER
from potentiel_solaire.features.roof_attributes import recuperation_mns, segmentation_toits

## Récupération du batiment

In [ ]:
priotirized_schools_buildings = gpd.read_file(RESULTS_FOLDER / "priotirized_schools_buildings.gpkg")

building = priotirized_schools_buildings[priotirized_schools_buildings["cleabs_bat"] == "BATIMENT0000000224614928"]

building.iloc[0]

In [ ]:
def show_raster(raster, **kwargs):
    """Show a raster with matplotlib."""
    fig, ax = plt.subplots(figsize=(12, 12))
    # Show the raster and add a colorbar to the plot
    img = show(raster, ax=ax, **kwargs)
    fig.colorbar(img.get_images()[0], ax=ax)

## MNS du batiment

In [ ]:
mns = recuperation_mns(building)

min_mns = mns[mns > -10].min()
max_mns = mns[mns > -10].max()

show_raster(mns, cmap="viridis", vmin=min_mns, vmax=max_mns)

## Segmentation du toit du batiment

In [ ]:
final_segment_toiture, slope_filtered, azimut_filtered, labeled_bounds = segmentation_toits(data=mns, debug=True)
final_segment_toiture.sort_values("surface", ascending=False)

### Pente

In [ ]:
show_raster(slope_filtered, cmap="viridis")

### Orientation

In [ ]:
show_raster(azimut_filtered, cmap="viridis")

In [ ]:
# Create a mapping from label to mean azimuth
label_to_azimut = dict(zip(final_segment_toiture["label"], final_segment_toiture["azimut"]))

# Prepare an array to hold azimuth values
azimut_map = np.full_like(labeled_bounds, np.nan, dtype=float)

# Replace label with mean azimuth
for label, azimut in label_to_azimut.items():
    azimut_map[labeled_bounds == label] = azimut

# Affichage des labels
plt.figure(figsize=(12, 12))
plt.imshow(azimut_map)
plt.colorbar(label="Azimuth moyen des clusters")
plt.show()

In [ ]:
# Create a mapping from label to mean slope
label_to_slope = dict(zip(final_segment_toiture["label"], final_segment_toiture["slope"]))

# Prepare an array to hold slope values
slope_map = np.full_like(labeled_bounds, np.nan, dtype=float)

# Replace label with mean slope
for label, slope in label_to_slope.items():
    slope_map[labeled_bounds == label] = slope

# Affichage des labels
plt.figure(figsize=(12, 12))
plt.imshow(slope_map)
plt.colorbar(label="Pente moyenne des clusters")
plt.show()

In [ ]:
geom = building.iloc[0].geometry
minx, miny, maxx, maxy = geom.bounds
transform = from_origin(minx, maxy, 0.5, 0.5)
crs = building.crs

with rasterio.open(
    RESULTS_FOLDER / f"azimut_{building.cleabs_bat}_map.tif",
    "w",
    driver="GTiff",
    height=azimut_map.shape[0],
    width=azimut_map.shape[1],
    count=1,
    dtype=azimut_map.dtype,
    crs=crs,
    transform=transform,
) as dst:
    dst.write(azimut_map, 1)